# Introduction

This notebook takes the monthly structured warrants report on Bursa Malaysia and applies some data wranggling to it:

See the [bm_report_converter](https://github.com/hsm207/bm_report_converter) repo for details.

# Libraries

In [1]:
import pandas as pd
from bfinance import Equity
import time
import numpy as np
from tqdm import tqdm

# Setup

In [2]:
%cd ..

/workspaces/bm_sector_rotation


Path to the monthly report:

In [3]:
DATA_FILE = "./data/2021-02_bm_monthly_sw.csv"

Path to save the results:

In [4]:
OUT_FILE = "./data/2021-02_bm_monthly_sw_processed.feather"

To enable tqdm with pandas:

In [5]:
tqdm.pandas()

# Data

In [6]:
column_names = ["No.", 
                "Stock Code", 
                "Stock Name", 
                "Security Type", 
                "No. of Warrants Outstanding", 
                "Maturity Date", 
                "Exercise Price", 
                "Conversion Ratio", 
                "Share per Warrant", 
                "Premium (Discount)", 
                "Gearing",
                "dummy"]

df = pd.read_csv(DATA_FILE, 
                 header = 0,
                 names = column_names,
                 index_col = "Stock Code",
                 usecols = lambda x: x != 'dummy',
                 na_values = '-',
                 thousands = ',',
                 parse_dates = ["Maturity Date"],
                 infer_datetime_format = False,
                 date_parser = lambda x: pd.to_datetime(x, format = "%d/%m/%Y"),
                 dtype = {
                     'Gearing': 'float'
                 })

df.head(10)

,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing
Stock Code,,,,,,,,,,
070364,1,A50CHIN-C64,Call WARRANTS,29000000,2021-03-31,HKD 20.000,1.5 for 1,0.67,NaN,NaN
070366,2,A50CHIN-C66,Call WARRANTS,24000000,2021-03-31,HKD 18.500,1.5 for 1,0.67,NaN,NaN
070368,3,A50CHIN-C68,Call WARRANTS,26000000,2021-06-30,HKD 21.500,1.5 for 1,0.67,NaN,NaN
070370,4,A50CHIN-C70,Call WARRANTS,28000000,2021-06-30,HKD 19.500,1.5 for 1,0.67,NaN,NaN
070372,5,A50CHIN-C72,Call WARRANTS,26000000,2021-09-30,HKD 28.000,2 for 1,0.50,NaN,NaN
070374,6,A50CHIN-C74,Call WARRANTS,30000000,2021-09-30,HKD 25.000,2 for 1,0.50,NaN,NaN
070349,7,A50CHIN-H49 #,PUT WARRANTS,32000000,2021-03-31,HKD 18.000,1.5 for 1,0.67,NaN,NaN
070351,8,A50CHIN-H51 #,PUT WARRANTS,30000000,2021-03-31,HKD 14.500,1.5 for 1,0.67,NaN,NaN
070353,9,A50CHIN-H53 #,PUT WARRANTS,33000000,2021-06-30,HKD 17.000,1.5 for 1,0.67,NaN,NaN


# Sanity Checks

Check that conversion from pdf has extracted all the warrants by looking at the sequence of the `No.` column:

In [7]:
assert df["No."].diff().dropna().sum() == df.shape[0] - 1

# Feature Engineering

Add some useful features for later analysis.

## Days to Expiry

Calculate how many days to expiry and ignore warrants with less than 60 days to expiry:

In [8]:
current_date = pd.Timestamp(pd.Timestamp.now().date())

df["time_to_expiry"] = df["Maturity Date"] - current_date
df = df.query('time_to_expiry > @pd.Timedelta(60, "day")')

num_warrants, _ = df.shape
print(f"There are {num_warrants:d} warrants with more than 60 days to expiry")

There are 578 warrants with more than 60 days to expiry


## Warrant Type

Simplify warrant type i.e. call or put:

In [9]:
df.loc[:, "Security Type"] = df["Security Type"].str.split(' ').str[0].str.lower()

In [10]:
df.sample(n = 10)

,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing,time_to_expiry
Stock Code,,,,,,,,,,,
9059CN,911,TSH-CN,call,35000000,2021-09-06,1.300,2 for 1,0.50,38.68%,6.24,161 days
521633,198,DSONIC-C33,call,78000000,2021-06-11,0.600,2.5 for 1,0.40,31.07%,6.87,74 days
0651FR,406,HSI-CFR,call,44000000,2021-07-29,27600.000,1200 for 1,0.00,NaN,NaN,122 days
522522,451,IHH-C22,call,150000000,2021-08-30,5.150,5 for 1,0.20,9.25%,12.70,154 days
7084CT,713,QL-CT,call,35000000,2021-05-31,6.667,8 for 1,0.13,25.97%,5.69,63 days
6633CX,525,LHI-CX,call,100000000,2021-11-30,0.750,2 for 1,0.50,28.57%,4.67,246 days
858333,539,MAHSING-C33,call,80000000,2021-10-21,1.050,5 for 1,0.20,58.23%,3.95,206 days
0650AT,248,FBMKLCI-HAT #,put,100000000,2021-07-30,1470.000,1000 for 1,0.00,NaN,NaN,123 days
0651FE,393,HSI-CFE,call,150000000,2021-05-31,25700.000,3500 for 1,0.00,NaN,NaN,63 days


## Warrant Profile

Get the profile of each warrant:

In [11]:
import traceback

def random_sleep():
    secs = np.random.randint(2, 10)
    time.sleep(secs)
    
def get_profile(stock_code):
    try:
        random_sleep()
        return Equity(stock_code).instrument_profile
    except Exception as e:
        print(f"Failed to process {stock_code}!")
        traceback.print_exc()

In [12]:
%%time
warrant_profiles = df.index.to_series().progress_apply(get_profile).to_list()
warrant_profiles = pd.DataFrame(warrant_profiles, index = df.index)

df = df.join(warrant_profiles)
df.sample(n = 10)

100%|██████████| 578/578 [59:02<00:00,  6.13s/it]  

CPU times: user 20.1 s, sys: 2 s, total: 22.1 s
Wall time: 59min 3s


,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing,time_to_expiry,underlying,issuer,market,sector
Stock Code,,,,,,,,,,,,,,,
518360,674,PCHEM-C60,call,60000000,2021-07-28,6.200,10 for 1,0.10,11.04%,3.62,121 days,PETRONAS CHEMICALS GROUP BERHAD,RHB,Structured Warrants,INDUSTRIAL PRODUCTS & SERVICES
381631,593,MISC-C31,call,60000000,2021-08-27,7.500,9 for 1,0.11,19.25%,11.59,151 days,MISC BERHAD,RHB,Structured Warrants,TRANSPORTATION & LOGISTICS
526437,543,MALAKOFC37,call,40000000,2021-08-26,1.050,4 for 1,0.25,34.57%,20.25,150 days,MALAKOFF CORPORATION BERHAD,RHB,Structured Warrants,UTILITIES
219420,594,MMCCORPC20,call,35000000,2021-06-28,1.000,1.5 for 1,0.67,41.67%,7.43,91 days,MMC CORPORATION BERHAD,KIBB,Structured Warrants,TRANSPORTATION & LOGISTICS
01381E,629,MYEG-C1E,call,100000000,2021-06-30,1.580,3 for 1,0.33,4.25%,3.37,93 days,MY E.G. SERVICES BERHAD,MIBB,Structured Warrants,TECHNOLOGY
696345,951,VS-C45,call,78000000,2021-09-08,2.380,5 for 1,0.20,14.33%,2.86,163 days,V.S. INDUSTRY BERHAD,MACQ,Structured Warrants,INDUSTRIAL PRODUCTS & SERVICES
5286CN,583,MI-CN,call,35000000,2021-05-31,2.880,4 for 1,0.25,3.03%,2.46,63 days,MI TECHNOVATION BERHAD,KIBB,Structured Warrants,TECHNOLOGY
527916,739,SERBADK-C16,call,100000000,2021-09-30,1.680,4 for 1,0.25,14.62%,6.11,185 days,SERBA DINAMIK HOLDINGS BERHAD,MIBB,Structured Warrants,ENERGY
0650DC,257,FBMKLCI-HDC #,put,14000000,2021-07-30,1600.000,300 for 1,0.00,NaN,NaN,123 days,FTSE BURSA MALAYSIA KLCI INDEX,MACQ,Structured Warrants,STRUCTURED WARRANTS


# Save Results 

Save the file:

In [13]:
df.reset_index().to_feather(OUT_FILE)

Re-read just to be sure:

In [14]:
pd.read_feather(OUT_FILE)\
    .set_index("Stock Code")\
    .sample(n=10)

,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing,time_to_expiry,underlying,issuer,market,sector
Stock Code,,,,,,,,,,,,,,,
5168HB,372,HARTA-HB #,put,100000000,2021-05-31,12.800,50 for 1,0.02,19.50%,2.11,63 days,HARTALEGA HOLDINGS BERHAD,MIBB,Structured Warrants,HEALTH CARE
2488C6,15,ABMB-C6,call,60000000,2021-08-27,2.700,4.5 for 1,0.22,14.62%,7.85,151 days,ALLIANCE BANK MALAYSIA BERHAD,RHB,Structured Warrants,FINANCIAL SERVICES
06509Z,234,FBMKLCI-C9Z,call,150000000,2021-06-30,1490.000,500 for 1,0.00,None,NaN,93 days,FTSE BURSA MALAYSIA KLCI INDEX,CIMB,Structured Warrants,STRUCTURED WARRANTS
514111,130,DAYANG-C11,call,35000000,2021-06-28,1.380,5 for 1,0.20,16.23%,4.03,91 days,DAYANG ENTERPRISE HOLDINGS BERHAD,KIBB,Structured Warrants,ENERGY
219420,594,MMCCORPC20,call,35000000,2021-06-28,1.000,1.5 for 1,0.67,41.67%,7.43,91 days,MMC CORPORATION BERHAD,KIBB,Structured Warrants,TRANSPORTATION & LOGISTICS
486378,869,TM-C78,call,100000000,2021-12-17,6.080,10 for 1,0.10,19.38%,4.91,263 days,TELEKOM MALAYSIA BHD,MIBB,Structured Warrants,TELECOMMUNICATIONS & MEDIA
467738,966,YTL-C38,call,23000000,2021-06-30,0.850,1.8 for 1,0.56,35.61%,14.67,93 days,YTL CORPORATION BERHAD,MACQ,Structured Warrants,UTILITIES
5161C8,493,JCY-C8,call,100000000,2021-09-30,0.920,3 for 1,0.33,144.30%,8.78,185 days,JCY INTERNATIONAL BERHAD,MIBB,Structured Warrants,TECHNOLOGY
4731CK,733,SCIENTX-CK,call,40000000,2021-08-26,3.600,15 for 1,0.07,43.66%,1.85,150 days,SCIENTEX BERHAD,RHB,Structured Warrants,INDUSTRIAL PRODUCTS & SERVICES
